#GTSRB DATA PROCESSING

In [ ]:
!unzip GTSRB_Final_Training_Images.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: GTSRB/Final_Training/Images/00035/00000_00019.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00020.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00021.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00022.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00023.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00024.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00025.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00026.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00027.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00028.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00000_00029.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00000.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00001.ppm  
  inflating: GTSRB/Final_Training/Images/00035/00001_00002.ppm  
  inflating: GTSRB/Final_Training

In [ ]:
import os
import pandas as pd
import shutil
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/Colab Notebooks/traffic

/content/gdrive/My Drive/Colab Notebooks/traffic


#Cloning darknet

In [ ]:
!git clone 'https://github.com/AlexeyAB/darknet.git'

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 8.95 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.
Updating files: 100% (2056/2056), done.


#Thay đổi file makefile để sử dụng GPU

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/gdrive/My Drive/Colab Notebooks/traffic/darknet


In [ ]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


#Tạo file thực thi và thư viện từ mã nguồn

In [ ]:
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘buff’ []8;;https://gcc.g

In [ ]:
!./darknet

usage: ./darknet <function>


In [ ]:
%cd ../../../../

/content


In [ ]:
os_dir ='gdrive/My Drive/Colab Notebooks/traffic/GTSRB/Final_Training/Images/'

#Đổi tên file ảnh và các giá trị tương ứng trong file csv

In [ ]:
for dir in os.listdir(os_dir):
    if dir == '.DS_Store' :
        continue
    inner_dir = os.path.join(os_dir, dir)
    print(inner_dir[-5:])
    for img in os.listdir(inner_dir):
        if img == "GT-" + dir + '.csv':
            csv_file = pd.read_csv(os.path.join(inner_dir,"GT-" + dir + '.csv'), sep=';')
            csv_file['Filename'] = csv_file['Filename'].apply(lambda x: f'{inner_dir[-5:]}_{x}')
            csv_file.to_csv(f'{inner_dir}/GGT-{dir}.csv', sep=';',index = False)
        else:
            continue
            os.rename(inner_dir + '/' + img, inner_dir + '/' + f'{inner_dir[-5:]}_{img}')

00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019
00020
00021
00022
00023
00024
00025
00026
00027
00028
00029
00030
00031
00032
00033
00034
00035
00036
00037
00038
00039
00040
00041
00042


#Copy vào 1 thư mục

In [ ]:
for dir in os.listdir(os_dir):
    if dir == '.DS_Store':
        continue
    inner_dir = os.path.join(os_dir, dir)
    print(inner_dir[-5:])
    for img in os.listdir(inner_dir):
        if img == "GT-" + dir + '.csv' or img == "GGT-" + dir + '.csv' or img == '00000_GT-00000.gsheet': # omit all files except the images
            continue
        else:
            shutil.copy(f'{inner_dir}/{img}','gdrive/My Drive/Colab Notebooks/traffic/TFinal')



00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019
00020
00021
00022
00023
00024
00025
00026
00027
00028
00029
00030
00031
00032
00033
00034
00035
00036
00037
00038
00039
00040
00041
00042


#Gộp tất cả file csv thành 1 file

In [ ]:
train_csv = pd.DataFrame()
for dir in os.listdir(os_dir):
    if dir == '.DS_Store' :
        continue
    inner_dir = os.path.join(os_dir, dir)
    csv_file = pd.read_csv(os.path.join(inner_dir,"GGT-" + dir + '.csv'), sep=';')

    train_csv = train_csv.append(csv_file)
print(train_csv.shape)

<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_csv = train_csv.append(csv_file)
<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_csv = train_csv.append(csv_file)
<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_csv = train_csv.append(csv_file)
<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_csv = train_csv.append(csv_file)
<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

(39209, 8)


<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_csv = train_csv.append(csv_file)
<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_csv = train_csv.append(csv_file)
<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_csv = train_csv.append(csv_file)
<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_csv = train_csv.append(csv_file)
<ipython-input-22-cbfbabe46123>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

#Tạo file annotation cho mỗi hình ảnh

Định dạng file annotation của yolov5: object-class/tọa độ x tâm object/ tọa độ y tâm object/chiều rộng object/chiều cao object

In [ ]:
train_csv.head()

,Filename,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
0,00000_00000_00000.ppm,29,30,5,6,24,25,0
1,00000_00000_00001.ppm,30,30,5,5,25,25,0
2,00000_00000_00002.ppm,30,30,5,5,25,25,0
3,00000_00000_00003.ppm,31,31,5,5,26,26,0
4,00000_00000_00004.ppm,30,32,5,6,25,26,0


In [ ]:
IMG_PATH = 'gdrive/My Drive/Colab Notebooks/traffic/TFinal'

In [ ]:
%cd gdrive/My Drive/Colab Notebooks/traffic/TFinal

/content/gdrive/My Drive/Colab Notebooks/traffic/TFinal


In [ ]:
def compare(v1, v2):
    if v1 > v2:
        vmax, vmin = v1, v2
        return vmax, vmin
    else:
        vmax, vmin = v2, v1
        return vmax, vmin
def convert_labels(z):
    x1 = z['Roi.X1']
    y1 = z['Roi.Y1']
    x2 = z['Roi.X2']
    y2 = z['Roi.Y2']
    size = [z['Height'],z['Width']]
    xmax, xmin = compare(x1, x2)
    ymax, ymin = compare(y1, y2)
    dw = 1./size[1]
    dh = 1./size[0]
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return x,y,w,h

In [ ]:
for index, x in train_csv.iterrows():
    name = x['Filename'].replace('.ppm','.txt')
    labels = convert_labels(x)
    cls = x['ClassId']
    f = open(f'{name}','w+')
    f.write(f'{cls} {labels[0]} {labels[1]} {labels[2]} {labels[3]}')
    f.close()

#Chia bộ dữ liệu thành Training Set và Testing Set

In [ ]:
y = train_csv['ClassId']

In [ ]:
from sklearn.model_selection import train_test_split
train_ds, valid_ds = train_test_split(
    train_csv,
    test_size = 0.3,
    random_state = 43,
    shuffle = True,
    stratify = y
)

In [ ]:
train_ds.reset_index()
valid_ds.reset_index()

,index,Filename,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
0,505,00023_00016_00025.ppm,69,68,6,7,63,62,23
1,541,00004_00018_00001.ppm,37,39,5,5,31,33,4
2,960,00011_00032_00000.ppm,35,35,6,5,30,30,11
3,946,00011_00031_00016.ppm,63,59,5,6,58,54,11
4,332,00013_00011_00002.ppm,38,39,6,5,33,34,13
...,...,...,...,...,...,...,...,...,...
11758,485,00005_00016_00005.ppm,34,34,6,6,29,29,5
11759,709,00017_00023_00019.ppm,39,41,5,6,34,36,17
11760,962,00018_00032_00002.ppm,48,48,6,6,43,43,18
11761,192,00042_00006_00012.ppm,36,37,5,6,31,32,42


#Tạo file txt chứa đường dẫn của Training Set và Testing Set

In [ ]:
f = open(f'train.txt','w+')
for index, x in train_ds.iterrows():
    name = x['Filename']
    f.write(f'TFinal/{name}\n')
f.close()

In [ ]:
f = open(f'test.txt','w+')
for index, x in valid_ds.iterrows():
    name = x['Filename']
    f.write(f'TFinal/{name}\n')
f.close()

In [ ]:
!pwd

/content/gdrive/MyDrive/Colab Notebooks/traffic/TFinal


In [ ]:
%cd ..

/content/gdrive/MyDrive/Colab Notebooks/traffic


#Tạo các nhãn

In [ ]:
## classes.names ##

classes = [
    'Speed limit (20km/h)',
    'Speed limit (30km/h)',
    'Speed limit (50km/h)',
    'Speed limit (60km/h)',
    'Speed limit (70km/h)',
    'Speed limit (80km/h)',
    'Speed limit (100km/h)',
    'Speed limit (120km/h)',
    'No passing',
    'No passing for vechiles over 3.5 metric tons',
    'No vehicles',
    'Vechiles over 3.5 metric tons prohibited',
    'Right-of-way at the next intersection',
    'General caution',
    'Dangerous curve to the left',
    'Dangerous curve to the right',
    'Double curve',
    'Bumpy road',
    'Slippery road',
    'Road narrows on the right',
    'Road work',
    'Traffic signals',
    'Pedestrians',
    'Children crossing',
    'Bicycles crossing',
    'Beware of ice/snow',
    'Wild animals crossing',
    'Turn right ahead',
    'Turn left ahead',
    'Ahead only',
    'Go straight or right',
    'Go straight or left',
    'Keep right',
    'Keep left',
    'Roundabout mandatory',
    'Priority road',
    'Yield',
    'Stop',
    'No entry',
    'End of speed limit (80km/h)',
    'End of all speed and passing limits',
    'End of no passing',
    'End of no passing by vechiles over 3.5 metric tons'
]

f = open(f'classes.names','w+')
for c in classes:
    f.write(f'{c}\n')
f.close()

## labelled_data.data ##

config = {
    'classes': 43,
    'train': 'TFinal/train.text',
    'valid': 'TFinal/valid.text',
    'names': 'classes.names',
    'backup': 'backup'
}
f = open(f'labelled_data.data', 'w+')
for key in config:
    f.write(f'{key} = {config[key]}\n')
f.close()

In [ ]:
drive.flush_and_unmount()